In [6]:
# Librerias
import tensorflow as tf
from tensorflow.keras import layers, models, datasets
import numpy as np
from PIL import Image
from google.colab import files

print("TensorFlow:", tf.__version__)

#Carga CIFAR-10
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

# Convertir de uint8 a float32
x_train = tf.cast(x_train, tf.float32) / 255.0
x_test = tf.cast(x_test, tf.float32) / 255.0

# Construimos el Dataset
BATCH_SIZE = 32  # pequeño para no saturar la RAM
AUTOTUNE = tf.data.AUTOTUNE

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.shuffle(10000).batch(BATCH_SIZE).prefetch(AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = test_ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)

# Modelo CNN pequeño porque solo tengo 16 RAM
def create_small_cnn():
    model = models.Sequential([
        layers.Input(shape=(32, 32, 3)),

        layers.Conv2D(32, (3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D(),

        layers.Conv2D(64, (3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D(),

        layers.Conv2D(128, (3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D(),

        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(10, activation="softmax")
    ])
    return model

model = create_small_cnn()
model.summary()

#Compilamos
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Entrenamos
EPOCHS = 5
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=test_ds
)

#Evaluación del modelo
loss, acc = model.evaluate(test_ds)
print("Precisión en test:", acc)


# Clasificación de imagen externa

# Clases de CIFAR-10
class_names = [
    "avión", "automóvil", "pájaro", "gato", "ciervo",
    "perro", "rana", "caballo", "barco", "camión"
]

# Preprocesamiento igual que el dataset
def preprocess_image(img_path):
    img = Image.open(img_path).convert("RGB")
    img = img.resize((32, 32))  # CIFAR-10 es 32x32
    img = np.array(img).astype("float32") / 255.0
    img = np.expand_dims(img, axis=0)
    return img

# Función para cargar y predecir
def predict_image():
    uploaded = files.upload()
    for name in uploaded.keys():
        print("Imagen cargada:", name)
        img = preprocess_image(name)
        pred = model.predict(img)
        class_id = np.argmax(pred)
        print("Predicción:", class_names[class_id])


TensorFlow: 2.19.0


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 356,810 (1.36 MB)

 Trainable params: 356,810 (1.36 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 117s 73ms/step - accuracy: 0.3926 - loss: 1.6563 - val_accuracy: 0.6172 - val_loss: 1.0882
Epoch 2/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 141s 73ms/step - accuracy: 0.6384 - loss: 1.0272 - val_accuracy: 0.6594 - val_loss: 0.9771
Epoch 3/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 141s 73ms/step - accuracy: 0.7060 - loss: 0.8346 - val_accuracy: 0.7199 - val_loss: 0.8148
Epoch 4/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 113s 73ms/step - accuracy: 0.7483 - loss: 0.7147 - val_accuracy: 0.7181 - val_loss: 0.8239
Epoch 5/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 145s 75ms/step - accuracy: 0.7788 - loss: 0.6284 - val_accuracy: 0.7274 - val_loss: 0.8204
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.7280 - loss: 0.8104
Precisión en test: 0.727400004863739


In [11]:
predict_image()

Saving camion.webp to camion (1).webp
Imagen cargada: camion (1).webp
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Predicción: perro


In [6]:
# Librerías
import tensorflow as tf
from tensorflow.keras import layers, models, datasets
import numpy as np
from PIL import Image
from google.colab import files

print("TensorFlow:", tf.__version__)

# Carga Fashion-MNIST
(x_train, y_train), (x_test, y_test) = datasets.fashion_mnist.load_data()

# Convertir de uint8 a float32 y normalizar
x_train = tf.cast(x_train, tf.float32) / 255.0
x_test  = tf.cast(x_test, tf.float32) / 255.0

# Expandir canal (Fashion-MNIST es gris)
x_train = tf.expand_dims(x_train, axis=-1)   # → (28,28,-1)
x_test  = tf.expand_dims(x_test, axis=-1)

# Construcción de datasets
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.shuffle(10000).batch(BATCH_SIZE).prefetch(AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = test_ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)

# Modelo CNN para Fashion-MNIST (28x28x1)
def create_small_cnn():
    model = models.Sequential([
        layers.Input(shape=(28, 28, 1)),

        layers.Conv2D(32, (3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D(),

        layers.Conv2D(64, (3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D(),

        layers.Conv2D(128, (3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D(),

        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(10, activation="softmax")
    ])
    return model

model = create_small_cnn()
model.summary()

# Compilación
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Entrenamiento
EPOCHS = 10
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=test_ds
)

# Evaluación
loss, acc = model.evaluate(test_ds)
print("Precisión en test:", acc)

# Clases de Fashion-MNIST
class_names = [
    "Camiseta/Top",
    "Pantalón",
    "Suéter",
    "Vestido",
    "Abrigo",
    "Sandalia",
    "Camisa",
    "Zapatilla",
    "Bolso",
    "Bota"
]

# Preprocesamiento para imágenes externas
def preprocess_image(img_path):
    img = Image.open(img_path).convert("L")  # Convertir a escala de grises
    img = img.resize((28, 28))
    img = np.array(img).astype("float32") / 255.0
    img = np.expand_dims(img, axis=-1)  # Canal
    img = np.expand_dims(img, axis=0)   # Batch 1
    return img

# Función para cargar y predecir una imagen externa
def predict_image():
    uploaded = files.upload()
    for name in uploaded.keys():
        print("Imagen cargada:", name)
        img = preprocess_image(name)
        pred = model.predict(img)
        class_id = np.argmax(pred)
        print("Predicción:", class_names[class_id])



TensorFlow: 2.19.0


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 14, 14, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 7, 7, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 241,546 (943.54 KB)

 Trainable params: 241,546 (943.54 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 129s 68ms/step - accuracy: 0.7743 - loss: 0.6194 - val_accuracy: 0.8798 - val_loss: 0.3305
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 125s 67ms/step - accuracy: 0.8944 - loss: 0.2869 - val_accuracy: 0.8973 - val_loss: 0.2810
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 121s 64ms/step - accuracy: 0.9137 - loss: 0.2318 - val_accuracy: 0.9056 - val_loss: 0.2486
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 133s 71ms/step - accuracy: 0.9239 - loss: 0.2030 - val_accuracy: 0.9053 - val_loss: 0.2699
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 121s 64ms/step - accuracy: 0.9338 - loss: 0.1775 - val_accuracy: 0.9062 - val_loss: 0.2661
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 125s 67ms/step - accuracy: 0.9451 - loss: 0.1512 - val_accuracy: 0.9162 - val_loss: 0.2494
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 120s 64ms/step - accuracy: 0.9510 - loss: 0.1312 - val_accuracy: 0.9096 - val_loss: 0.2699
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 120s 64ms/step - accuracy: 

###**Pueba on mas epoas**

In [1]:
# Librerías
import tensorflow as tf
from tensorflow.keras import layers, models, datasets
import numpy as np
from PIL import Image
from google.colab import files

print("TensorFlow:", tf.__version__)

# Carga Fashion-MNIST
(x_train, y_train), (x_test, y_test) = datasets.fashion_mnist.load_data()

# Convertir de uint8 a float32 y normalizar
x_train = tf.cast(x_train, tf.float32) / 255.0
x_test  = tf.cast(x_test, tf.float32) / 255.0

# Expandir canal (Fashion-MNIST es gris)
x_train = tf.expand_dims(x_train, axis=-1)   # → (28,28,1)
x_test  = tf.expand_dims(x_test, axis=-1)

# Construcción de datasets
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.shuffle(10000).batch(BATCH_SIZE).prefetch(AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = test_ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)

# Modelo CNN para Fashion-MNIST (28x28x1)
def create_small_cnn():
    model = models.Sequential([
        layers.Input(shape=(28, 28, 1)),

        layers.Conv2D(32, (3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D(),

        layers.Conv2D(64, (3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D(),

        layers.Conv2D(128, (3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D(),

        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(10, activation="softmax")
    ])
    return model

model = create_small_cnn()
model.summary()

# Compilación
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Entrenamiento
EPOCHS = 15
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=test_ds
)

# Evaluación
loss, acc = model.evaluate(test_ds)
print("Precisión en test:", acc)

# Clases de Fashion-MNIST
class_names = [
    "Camiseta/Top",
    "Pantalón",
    "Suéter",
    "Vestido",
    "Abrigo",
    "Sandalia",
    "Camisa",
    "Zapatilla",
    "Bolso",
    "Bota"
]

# Preprocesamiento para imágenes externas
def preprocess_image(img_path):
    img = Image.open(img_path).convert("L")  # Convertir a escala de grises
    img = img.resize((28, 28))
    img = np.array(img).astype("float32") / 255.0
    img = np.expand_dims(img, axis=-1)  # Canal
    img = np.expand_dims(img, axis=0)   # Batch 1
    return img

# Función para cargar y predecir una imagen externa
def predict_image():
    uploaded = files.upload()
    for name in uploaded.keys():
        print("Imagen cargada:", name)
        img = preprocess_image(name)
        pred = model.predict(img)
        class_id = np.argmax(pred)
        print("Predicción:", class_names[class_id])



TensorFlow: 2.19.0
29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 14, 14, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 7, 7, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 241,546 (943.54 KB)

 Trainable params: 241,546 (943.54 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 104s 54ms/step - accuracy: 0.7806 - loss: 0.6088 - val_accuracy: 0.8797 - val_loss: 0.3289
Epoch 2/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 99s 53ms/step - accuracy: 0.8952 - loss: 0.2872 - val_accuracy: 0.8964 - val_loss: 0.2798
Epoch 3/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 98s 52ms/step - accuracy: 0.9131 - loss: 0.2337 - val_accuracy: 0.8970 - val_loss: 0.2839
Epoch 4/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 99s 53ms/step - accuracy: 0.9248 - loss: 0.2005 - val_accuracy: 0.9087 - val_loss: 0.2398
Epoch 5/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 140s 52ms/step - accuracy: 0.9360 - loss: 0.1706 - val_accuracy: 0.9146 - val_loss: 0.2437
Epoch 6/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 98s 52ms/step - accuracy: 0.9457 - loss: 0.1487 - val_accuracy: 0.9188 - val_loss: 0.2363
Epoch 7/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 142s 52ms/step - accuracy: 0.9525 - loss: 0.1281 - val_accuracy: 0.9164 - val_loss: 0.2473
Epoch 8/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 142s 52ms/step - accuracy: 0.95